# Use a free public proxy list for python requests
## Ref: https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/

# Useful proxy list websites
1. https://www.sslproxies.org/ (100 IPs all Https, tested)
2. http://spys.one/en/https-ssl-proxy/ (90 IPs which are https, NOT TESTED YET)
3. https://hidemy.name/en/proxy-list/?type=hs#list (Over 2000 https, NOT TESTED YET)
4. https://www.proxy-list.download/HTTPS (1500 https, NOT TESTED YET)

In [2]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import threading
# import urllib2
import time

# Generate a random user agent

In [64]:
country_list = ['United States', 'United Kingdom', 'Belarus', 'Czech Republic', 'Spain',
               'Brazil', 'France', 'Canada', 'Poland', 'Armenia', 'Ukraine', 'France',
               'Mexico', 'Georgia', 'Hungary']

In [65]:
user_agent = UserAgent() 
proxies_no_filter = [] 

# Set the URL of the proxy list

In [66]:
proxy_website = 'https://www.sslproxies.org/'

# Retrieve latest proxies

In [67]:
# Retrieve latest proxies
proxies_req = Request(proxy_website)
proxies_req.add_header('User-Agent', user_agent.random)
proxies_doc = urlopen(proxies_req).read().decode('utf8')

soup = BeautifulSoup(proxies_doc, 'html.parser')
proxies_table = soup.find(id='proxylisttable')

In [68]:
for row in proxies_table.tbody.find_all('tr'):
#     print(row.find_all('td')[3].string)
    proxies_no_filter.append({
      'ip':    row.find_all('td')[0].string,
      'port':  row.find_all('td')[1].string,
      'country': row.find_all('td')[3].string,
      'https': row.find_all('td')[6].string})
print(proxies_no_filter[0])

{'ip': '14.162.145.116', 'port': '46516', 'country': 'Vietnam', 'https': 'yes'}


In [69]:
# Filter the proxies with the conditions we need
proxies = []

for proxy_ip in proxies_no_filter:
    if (proxy_ip['country'] in country_list) and (proxy_ip['https'] == 'yes'):
        proxies.append(proxy_ip)
    
    else:
        pass

In [70]:
# Count the number of total proxies
print(len(proxies))

42


In [71]:
print(proxies_no_filter)

[{'ip': '14.162.145.116', 'port': '46516', 'country': 'Vietnam', 'https': 'yes'}, {'ip': '37.187.149.234', 'port': '1080', 'country': 'France', 'https': 'yes'}, {'ip': '178.32.80.237', 'port': '8080', 'country': 'France', 'https': 'yes'}, {'ip': '150.129.56.138', 'port': '31111', 'country': 'Indonesia', 'https': 'yes'}, {'ip': '154.73.137.157', 'port': '43333', 'country': 'South Africa', 'https': 'yes'}, {'ip': '179.127.242.39', 'port': '31873', 'country': 'Brazil', 'https': 'yes'}, {'ip': '180.180.175.70', 'port': '59692', 'country': 'Thailand', 'https': 'yes'}, {'ip': '194.228.129.189', 'port': '30600', 'country': 'Czech Republic', 'https': 'yes'}, {'ip': '190.57.143.66', 'port': '50719', 'country': 'Ecuador', 'https': 'yes'}, {'ip': '88.12.42.81', 'port': '47617', 'country': 'Spain', 'https': 'yes'}, {'ip': '194.44.243.186', 'port': '47312', 'country': 'Ukraine', 'https': 'yes'}, {'ip': '152.204.128.46', 'port': '34562', 'country': 'Colombia', 'https': 'yes'}, {'ip': '182.16.162.89'

# Set the URL for testing

In [71]:
test_url = 'http://icanhazip.com'
nb_request = 50
step_for_random_proxy = 5

# Choose a random proxy

In [75]:
def random_proxy():
    return random.randint(0, len(proxies) - 1)

# Make parallel requests

In [57]:
from multiprocessing.pool import ThreadPool
from time import time as timer
from urllib.request import urlopen

## Get number of process and proxies we need

In [76]:
proxy_list = []
nb_thread = 2

for i in range(nb_thread):
    proxy_index = random_proxy()
    proxy = proxies[proxy_index]
    proxy_list.append(proxy)

In [80]:
print(proxy_list)

[{'ip': '5.77.254.146', 'port': '56955', 'country': 'Armenia', 'https': 'yes'}, {'ip': '194.228.129.189', 'port': '30600', 'country': 'Czech Republic', 'https': 'yes'}]


In [77]:
def test_proxy(proxy):
    for n in range(1, nb_request):
        req = Request(test_url)
        req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

        # Every 10 requests, generate a new proxy
        if n % step_for_random_proxy == 0:
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

        # Intercept broken proxies and delete them from the list and notice the user
        try:
            my_ip = urlopen(req).read().decode('utf8')
            print('#' + str(n) + ': ' + my_ip)
        except: # If error, delete this proxy and find another one
            del proxies[proxy_index]
            print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
            proxy_index = random_proxy()
            proxy = proxies[proxy_index]

In [81]:
ThreadPool(2).imap_unordered(test_proxy, proxy_list)

In [78]:
results = ThreadPool(2).imap_unordered(test_proxy, proxy_list)

In [79]:
print(results)

In [ ]:
proxy_index = random_proxy()
proxy = proxies[proxy_index]

for n in range(1, nb_request):
    req = Request(test_url)
    req.set_proxy(proxy['ip'] + ':' + proxy['port'], 'http')

    # Every 10 requests, generate a new proxy
    if n % step_for_random_proxy == 0:
        proxy_index = random_proxy()
        proxy = proxies[proxy_index]

    # Intercept broken proxies and delete them from the list and notice the user
    try:
        my_ip = urlopen(req).read().decode('utf8')
        print('#' + str(n) + ': ' + my_ip)
    except: # If error, delete this proxy and find another one
        del proxies[proxy_index]
        print('Proxy ' + proxy['ip'] + ':' + proxy['port'] + ' deleted.')
        proxy_index = random_proxy()
        proxy = proxies[proxy_index]

#1: 46.175.185.239

Proxy 46.175.185.239:53236 deleted.
#3: 195.230.115.241

#4: 195.230.115.241

#5: 195.230.115.241

Proxy 217.182.120.161:1080 deleted.
#7: 125.209.116.146

#8: 125.209.116.146

Proxy 125.209.116.146:46258 deleted.
#10: 114.199.115.46

#11: 125.26.109.114

#12: 125.26.109.114

#13: 125.26.109.114

#14: 125.26.109.114

#15: 125.26.109.114

Proxy 46.201.235.229:3128 deleted.
